In [34]:
from lovit_textmining_dataset.navernews_10days import get_news_paths
from soynlp.noun import LRNounExtractor_v2
from soynlp.utils import DoublespaceLineCorpus
corpus_path_1 = '2016-10-20.txt'

f1 = open(corpus_path_1)
lines = f1.readlines()
type(lines)



/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


list

In [35]:
from konlpy.tag import Komoran

komoran = Komoran()

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/konlpy/jvm.py:8: DeprecationWarning: This version of JPype is now deprecated. Please use this version instead: https://github.com/jpype-project/jpype
  import jpype


In [43]:
nouns = komoran.nouns(lines[3])
print(nouns)

['테헤란', '연합뉴스', '강훈', '특파원', '이용', '승객', '수', '기준', '세계', '최대', '공항', '아랍에미리트', '두바이', '국제공항', '일', '현지', '시간', '공항', '이륙', '항공기', '탑승객', '삼성전자', '갤럭시', '노트', '을', '휴대', '두바이', '국제공항', '항공', '관련', '기구', '권고', '안전', '우려', '스마트폰', '갤럭시', '노트', '을', '휴대', '비행기', '탑승', '검색', '중', '발견', '압수', '계획', '발표', '공항', '측', '갤럭시', '노트', '배터리', '폭발', '우려', '제기', '만큼', '제품', '공항', '안', '이용객', '당부', '조치', '두바이', '국제공항', '뿐', '공항', '두바이', '월드', '센터', '적용', '배터리', '폭발', '문제', '회수', '갤럭시', '노트', '연합뉴스', '자료', '사진']


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
type(corpus)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


soynlp.utils.utils.DoublespaceLineCorpus

In [50]:
corpus_path = '2016-10-20_komoran.txt'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
        #f = nouns
            for doc in f:
                yield doc.strip().split()
                #토크나이징

documents = Documents(corpus_path)
type(documents)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


__main__.Documents

Dictionary: 단어를 int형식으로 변환하는 encoder

In [51]:
import gensim

dictionary = gensim.corpora.Dictionary(documents)

print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dictionary size : 66172


In [52]:
type(dictionary)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


gensim.corpora.dictionary.Dictionary

In [53]:
from collections import Counter

min_count = 40
word_counter = Counter((word for words in documents for word in words))
removal_word_idxs = {
    dictionary.token2id[word] for word, count in word_counter.items()
    if count < min_count
}

dictionary.filter_tokens(removal_word_idxs)
dictionary.compactify()
print('dictionary size : %d' % len(dictionary)) # dictionary size : 10354

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dictionary size : 12072


In [54]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    if i >= 5: break
    print(doc)

[]
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
[(0, 2), (1, 4), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 2), (18, 5), (19, 1), (20, 2), (21, 1), (22, 20), (23, 5), (24, 3), (25, 1), (26, 17), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 4), (35, 1), (36, 1), (37, 1), (38, 1), (39, 3), (40, 3), (41, 1), (42, 1), (43, 1), (44, 5), (45, 11), (46, 3), (47, 11), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 7), (57, 1), (58, 3), (59, 5), (60, 1), (61, 2), (62, 5), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 4), (70, 20), (71, 1), (72, 2), (73, 33), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 7), (81, 1), (82, 4), (83, 2), (84, 1), (85, 1), (86, 5), (87, 2), (88, 1), (89, 2), (90, 8), (91, 2), (92, 2), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 4), (100, 19), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 5), (107, 2), (1

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
from lovit_textmining_dataset.navernews_10days import get_bow

x, index2word, word2index = get_bow(date='2016-10-20', tokenize='noun')

In [7]:
'''
import gensim
from gensim.corpora.dictionary import Dictionary

corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)
dictionary = Dictionary.from_corpus(
    corpus,
    id2word = dict(enumerate(index2word))
)
'''

'\nimport gensim\nfrom gensim.corpora.dictionary import Dictionary\n\ncorpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)\ndictionary = Dictionary.from_corpus(\n    corpus,\n    id2word = dict(enumerate(index2word))\n)\n'

In [8]:
'''
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=50)
with open(lda_model_path, 'wb') as f:
    pickle.dump(lda_model, f)
'''

"\nfrom gensim.models import LdaModel\n\nlda_model = LdaModel(corpus, id2word=dictionary, num_topics=50)\nwith open(lda_model_path, 'wb') as f:\n    pickle.dump(lda_model, f)\n"

In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [10]:
from gensim.models import LdaModel
lda_model = LdaModel(corpus, id2word=dictionary, num_topics=50)

In [11]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(50,)
0.9999999
(50, 12072)
1.0


In [13]:
import pyLDAvis.gensim_models as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
import pyLDAvis

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
pyLDAvis.display(prepared_data)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 영어 정제 데이터를 이용한 lda

In [56]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


11314

In [57]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-57-b4124dfb5e6a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [58]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [60]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.023*"jesus" + 0.011*"church" + 0.011*"book" + 0.010*"bible"')
(1, '0.018*"health" + 0.014*"medical" + 0.010*"disease" + 0.008*"pain"')
(2, '0.020*"space" + 0.007*"nasa" + 0.006*"system" + 0.005*"data"')
(3, '0.011*"strip" + 0.010*"darren" + 0.009*"promo" + 0.008*"copper"')
(4, '0.016*"cover" + 0.011*"copies" + 0.008*"captain" + 0.008*"annual"')
(5, '0.020*"would" + 0.011*"like" + 0.010*"think" + 0.010*"people"')
(6, '0.010*"file" + 0.009*"thanks" + 0.008*"please" + 0.008*"windows"')
(7, '0.018*"printf" + 0.012*"bytes" + 0.011*"tobacco" + 0.009*"part"')
(8, '0.013*"available" + 0.011*"window" + 0.009*"server" + 0.009*"version"')
(9, '0.015*"period" + 0.009*"picture" + 0.008*"scott" + 0.007*"smith"')
(10, '0.023*"armenian" + 0.018*"armenians" + 0.016*"turkish" + 0.010*"people"')
(11, '0.013*"bike" + 0.010*"cars" + 0.009*"engine" + 0.009*"price"')
(12, '0.031*"game" + 0.028*"team" + 0.022*"games" + 0.020*"play"')
(13, '0.016*"government" + 0.013*"people" + 0.012*"right" + 0.009*"ri

In [63]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [64]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(4, 0.086954154), (5, 0.530454), (9, 0.017533205), (10, 0.14165723), (17, 0.21109101)]
1 번째 문서의 topic 비율은 [(5, 0.7386767), (9, 0.027489737), (15, 0.18482758), (17, 0.027925095)]
2 번째 문서의 topic 비율은 [(5, 0.710004), (17, 0.2754392)]
3 번째 문서의 topic 비율은 [(0, 0.042405054), (5, 0.40123564), (6, 0.077281155), (7, 0.034448218), (16, 0.05930863), (17, 0.07261876), (19, 0.3025195)]
4 번째 문서의 topic 비율은 [(5, 0.36949483), (6, 0.11884889), (9, 0.039764576), (10, 0.19396304), (12, 0.25013742)]


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
